In [27]:
import pandas as pd
from slugify import slugify
import json

def formatINR(number):
    s, *d = str(number).partition(".")
    r = ",".join([s[x-2:x] for x in range(-3, -len(s), -2)][::-1] + [s[-3:]])
    return "".join([r] + d)

In [9]:
# read data

raw_data = pd.read_excel('/home/abhinav/sink/D4D/data.xlsx', engine='openpyxl')
indicators = pd.read_excel('/home/abhinav/sink/D4D/Indicators.xlsx', engine='openpyxl')
dist_meta_data = {"morigaon":{"Title": "Morigaon", "About": "Morigaon district is an administrative district in the state of Assam in India. The district headquarters is located at Morigaon. The ancient place of occult Mayong is located in this district as well as Pobitora Wildlife Sanctuary.", "link": "https://morigaon.gov.in/"}, "nagaon":{"Title": "Nagaon", "About": "Nagaon is an administrative district in the Indian state of Assam. The undivided district was the most populous in the state as per 2011 census. The eastern, western, and southern fringes of the newly organized district were once ruled by different small feudal kings or their agents. It was a newly organized village system-hence called “Nagaon”, Na meaning new.", "link": "https://nagaon.assam.gov.in/"} }
dept_meta_data = {'panchayat-and-rural-development': {'Title': "Panchayat and Rural Development", 'Desc':'The Panchayat & Rural Development Department largely works to enhance livelihood opportunities, address chronic poverty, provides social security and works for economic inclusion of rural poor families. The Panchayat & Rural Development Department has been implementing a number of programmes in the rural area of the State by empowering rural poor through power of rights based law, aiming to transform the geography of poverty in the State.', 'schemes':['mgnrega']}, 
             'public-health-engineering': {'Title': "Public Health Engineering", 'Desc':'The Public Health Engineering Department has a mission to enhance quality of life of the people by ensuring sustainable safe drinking water and sanitation facilities and services along with promoting hygiene practices according to their choices and affordability.', 'schemes':['jjm']}}
scheme_meta_data = {"mgnrega": {"Name": "Mahatma Gandhi National Rural Employment Guarantee Act (MGNREGA)", "Title": "MGNREGA", "Desc": "The objective of MGNREGS is to enhance livelihood security of the rural masses. The scheme is meant to provide at least 100 days of wage employment per financial year (FY) to every rural household whose adult members are willing to do unskilled manual work. This is one of the world’s largest public sector employment programmes that provides guaranteed income through employment.", "dept_slug":"panchayat-and-rural-development", "logo": "/logo/mgnrega.webp"},
                    "jjm": {"Name": "Jal Jeevan Mission (JJM)", "Title": "JJM", "Desc": "The National Rural Drinking Water Programme / Jal Jeevan Mission is an initiative of the Government of India under the Ministry of Jal Shakti. It aims to provide safe and adequate drinking water to all households in rural India through provision of individual household tap connections by the year 2024.", "dept_slug":"public-health-engineering", "logo": "/logo/nhm.png"}}

In [10]:
#Merge indicator data into data file 
data = pd.merge(raw_data,indicators[['indicator_slug','indicator_type', 'indicator_selected', 'indicator_overview', 'indicator_explorer', 'indicator_dept', 'indicator_district', 'indicator_unit']],on='indicator_slug', how='left')


In [11]:
indicators[indicators.indicator_district == 'yes']

,scheme_name,indicator_report,indicator_name,indicator_slug,indicator_type,indicator_name_short,indicator_selected,indicator_overview,indicator_explorer,indicator_dept,indicator_district,indicator_unit
0,JJM,B1- Basic Habitation Information,Habitations,bhinh,district_profile,No of habitations,yes,yes,yes,no,yes,number
1,JJM,B1- Basic Habitation Information,HouseHolds,bhnhh,district_profile,No of households,yes,yes,yes,no,yes,number
2,JJM,B1- Basic Habitation Information,Panchayats,bhinp,district_profile,No of panchayats,yes,no,no,no,yes,number
3,JJM,B1- Basic Habitation Information,Population,bhpop,district_profile,Population,yes,yes,yes,no,yes,number
87,MGNREGA,Category Wise Household Workers,No of active jobcards,noajc,district_profile,NaN,yes,no,yes,no,yes,number
118,MGNREGA,SC ST Employment Status,No of households with jobcards,nohwj,district_profile,NaN,yes,no,yes,no,yes,number
132,MGNREGA,SC ST Employment Status,No of families that completed 100 days of work,noftc,district_performance,NaN,yes,no,yes,no,yes,number
255,MGNREGA,Derived,Cost to Generate 1 rupee wage,cg1rw,district_performance,Cost to generate ₹1 wage,yes,yes,yes,no,yes,rupees


In [12]:

# filter data for district page indicators and structure the column values

data = data[data.indicator_district == 'yes']


data['district_name'] = data['district_name'].map(lambda x: x.title())
data['district_slug'] = data['district_name'].map(lambda x: slugify(x))
data['block_name'] = data['block_name'].map(lambda x: x.title())
data['block_slug'] = data['block_name'].map(lambda x: slugify(x))
data['scheme_slug'] = data['scheme_name'].map(lambda x: slugify(x))


data.head()

,db_code,district_name,district_code,block_name,block_code,financial_year,scheme_name,indicator_report,date_of_update,indicator_name,...,indicator_type,indicator_selected,indicator_overview,indicator_explorer,indicator_dept,indicator_district,indicator_unit,district_slug,block_slug,scheme_slug
140,27--1,Nagaon,27,Khagorijan,1,2023-2024,MGNREGA,Category Wise Household Workers,7-2023-09-34,No of active jobcards,...,district_profile,yes,no,yes,no,yes,number,nagaon,khagorijan,mgnrega
141,27--2,Nagaon,27,Batadraba,2,2023-2024,MGNREGA,Category Wise Household Workers,7-2023-09-34,No of active jobcards,...,district_profile,yes,no,yes,no,yes,number,nagaon,batadraba,mgnrega
142,27--3,Nagaon,27,Juria,3,2023-2024,MGNREGA,Category Wise Household Workers,7-2023-09-34,No of active jobcards,...,district_profile,yes,no,yes,no,yes,number,nagaon,juria,mgnrega
143,27--4,Nagaon,27,Kaliabor,4,2023-2024,MGNREGA,Category Wise Household Workers,7-2023-09-34,No of active jobcards,...,district_profile,yes,no,yes,no,yes,number,nagaon,kaliabor,mgnrega
144,27--5,Nagaon,27,Rupahihut,5,2023-2024,MGNREGA,Category Wise Household Workers,7-2023-09-34,No of active jobcards,...,district_profile,yes,no,yes,no,yes,number,nagaon,rupahihut,mgnrega


In [6]:
# calculate district aggregate values for district page 
def f(x):
    d = {}
    d['noajc'] = x[x['indicator_slug'] == 'noajc']['indicator_value'].sum()
    d['nohwj'] = x[x['indicator_slug'] == 'nohwj']['indicator_value'].sum()
    d['noftc'] = x[x['indicator_slug'] == 'noftc']['indicator_value'].sum()
    d['cg1rw'] = round(x[x['indicator_slug'] == 'cg1rw']['indicator_value'].mean(), 2)

    return pd.Series(d, index=['noajc', 'nohwj', 'noftc', 'cg1rw']) 

dist_data = data.groupby(['district_name', 'district_slug']).apply(f).reset_index()
dist_data.head()



,district_name,district_slug,noajc,nohwj,noftc,cg1rw
0,Morigaon,morigaon,319485.0,436337.0,291.0,1.15
1,Nagaon,nagaon,482540.0,679157.0,1302.0,1.12


In [29]:
# District Page Generator

base_json = {}

for index, row in dist_data.iterrows():
    
    # insert district
    if  row.district_slug not in base_json:
        base_json[row.district_slug] = {
            "title": row.district_name,
            "href": row.district_slug,
            "about": dist_meta_data[row.district_slug]['About'],
            "link": dist_meta_data[row.district_slug]['link'],
            "collapsible": {
            "title": "District Highlights & Information",
            "content": {
                "cards": [
                    {
                        "value": str(formatINR(int(row.noajc))),
                        "label": "No of active jobcards",
                        "color": "highlight"
                    },
                    {
                        "value": str(formatINR(int(row.nohwj))), 
                        "label": "No of households with jobcards",
                        "color": "highlight"
                    },
                    {
                        "value": str(formatINR(int(row.noftc))), 
                        "label": "No of families that completed 100 days of work",
                        "color": "highlight"
                    },
                    {
                        "value": "INR " + str(row.cg1rw),
                        "label": "Cost to Generate 1 rupee wage",
                        "color": "highlight"
                    }
                ],
                "rightTitle": "About" + row.district_name,
                "description": [dist_meta_data[row.district_slug]["About"] if row.district_slug in dist_meta_data else row.district_name]
            }
            },
            "listTitle": "Browse Line Departments",
            "list": [
            {
                "label": "Panchayat & Rural Development",
                "href": "panchayat-and-rural-development"
            },
            {
                "label": "Public Health Engineering",
                "href": "public-health-engineering"
            }
            ]
            }
        
            
    else:
        pass
        
with open('dist_data.json', 'w') as fp:
    json.dump(base_json, fp, indent=4)